<a href="https://colab.research.google.com/github/KazukiHirata-sun/ai_project_dev_2022/blob/main/Appendix/image_classifier_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model training and preservation
Build and train a [CNN model](https://viblo.asia/p/deep-learning-tim-hieu-ve-mang-tich-chap-cnn-maGK73bOKj2) for [image identification](https://www.analyticsvidhya.com/blog/2020/02/learn-image-classification-cnn-convolutional-neural-networks-3-datasets/) at Google Colaboratory.  
In this case, we will use [cifar-10](https://www.cs.toronto.edu/~kriz/cifar.html) as training data.  
The trained model is saved and downloaded.

[Refarence](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)

## Data Preprocessing
Load CIFAR-10 and configure DataLoader settings along with data expansion.  

In [ ]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

affine = transforms.RandomAffine([-15, 15], scale=(0.8, 1.2))  # Rotate and Resize
flip = transforms.RandomHorizontalFlip(p=0.5)  # right side flip
normalize = transforms.Normalize((0.0, 0.0, 0.0), (1.0, 1.0, 1.0))  # Mean is 0, standard deviation is 1
to_tensor = transforms.ToTensor()

transform_train = transforms.Compose([affine, flip, to_tensor, normalize])
transform_test = transforms.Compose([to_tensor, normalize])
cifar10_train = CIFAR10("./data", train=True, download=True, transform=transform_train)
cifar10_test = CIFAR10("./data", train=False, download=True, transform=transform_test)

# DataLoader settings
batch_size = 64
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(cifar10_test, batch_size=len(cifar10_test), shuffle=False)

## Fixed seeds

In [ ]:
import os
import random

import torch
import numpy as np

# Setup random seed
def set_seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

set_seed_everything(2022)

## Model Building
Build a model of CNN as a class that inherits from the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) module.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # Convolutional layer: (number of input channels, number of filters, filter size)
        self.pool = nn.MaxPool2d(2, 2)  # Pooling layer: (area size, stride)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 256)  # Fully Connected layer
        self.dropout = nn.Dropout(p=0.5)  # Dropout
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

net = Net()
net.cuda()  # GPU
print(net)

## Learning
Train the model.  
Set the cross-entropy error as the [loss function](https://www.datarobot.com/blog/introduction-to-loss-functions/#:~:text=Further%20reading-,What's%20a%20loss%20function%3F,ll%20output%20a%20lower%20number.) and Adam as the [optimization algorithm](https://d2l.ai/chapter_optimization/index.html). 
Training takes time, so select GPU in Edit → Hardware Accelerator in Notebook Settings.

In [ ]:
from torch import optim

# Cross Entropy
loss_fnc = nn.CrossEntropyLoss()

# Optimization Algorithm
optimizer = optim.Adam(net.parameters())

record_loss_train = []
record_loss_test = []

# Leaning
x_test, t_test = iter(test_loader).next()
x_test, t_test = x_test.cuda(), t_test.cuda()
for i in range(20):  # 20 epoc
    net.train()  # Train
    loss_train = 0
    for j, (x, t) in enumerate(train_loader):  # Mini Batch
        x, t = x.cuda(), t.cuda()  # GPU
        y = net(x)
        loss = loss_fnc(y, t)
        loss_train += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train /= j+1
    record_loss_train.append(loss_train)

    net.eval()  # evalation 
    y_test = net(x_test)
    loss_test = loss_fnc(y_test, t_test).item()
    record_loss_test.append(loss_test)

    if i%1 == 0:
        print("Epoch:", i, "Loss_Train:", loss_train, "Loss_Test:", loss_test)

## Error Trends
Displays graphs of error trends in training and test data.

[Refarence](https://matplotlib.org/) 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(record_loss_train)), record_loss_train, label="Train")
plt.plot(range(len(record_loss_test)), record_loss_test, label="Test")
plt.legend()

plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()

## Accuracy
To understand the performance of the model, we measure the percentage of correct answers using test data.

In [ ]:
correct = 0
total = 0
net.eval()  # Evaluation
for i, (x, t) in enumerate(test_loader):
    x, t = x.cuda(), t.cuda()  # GPU
    y = net(x)
    correct += (y.argmax(1) == t).sum().item()
    total += len(x)
print("Accuracy:", str(correct/total*100) + "%")

## Save model
Save the parameters of the trained model.    
Each parameter of the model can be obtained by `state_dict()`, so save it.

In [ ]:
import torch

for key in net.state_dict():
    print(key, ": ", net.state_dict()[key].size())
print(net.state_dict()["conv1.weight"][0]) 

# Save Model
torch.save(net.state_dict(), "model_cnn.pth")  